# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

## A*

In [3]:
import numpy as np
import time
import heapq

class Node:
    goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    def __init__(self, state, parent=None, action=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.g = g
        self.h = h
        self.f = self.g + self.h
        self.children = []

    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

    def __lt__(self, other):
        return self.f > other.f

    @staticmethod
    def heuristic(board):
        distance = 0
        for i in range(3):
            for j in range(3):
                if board[i][j] != 0:
                    value = board[i][j] - 1
                    target_row = value // 3
                    target_col = value % 3
                    distance += abs(i - target_row) + abs(j - target_col)
        return distance

    def expand_node(self):
        zero_positions = np.argwhere(self.state == 0)
        zero_i, zero_j = zero_positions[0]

        for new_i, new_j in [(zero_i - 1, zero_j), (zero_i + 1, zero_j), (zero_i, zero_j - 1), (zero_i, zero_j + 1)]:
            if 0 <= new_i < 3 and 0 <= new_j < 3:
                new_state = np.copy(self.state)
                new_state[zero_i, zero_j], new_state[new_i, new_j] = new_state[new_i, new_j], new_state[zero_i, zero_j]
                new_node = Node(new_state, parent=self, action=f"({new_i},{new_j})", g=self.g + 1, h=Node.heuristic(new_state))
                self.children.append(new_node)

def aPathFinder(board):
    start = time.time()
    goal = Node.goal_state
    visited = set()
    open_list = [(board.f, board)]
    heapq.heapify(open_list)
    count = 0

    while open_list:
        count += 1
        current_node = heapq.heappop(open_list)[1]

        if np.array_equal(current_node.state, goal):
            path = []
            while current_node:
                path.append(current_node.state)
                current_node = current_node.parent
            end = time.time()
            execution_time = end - start
            return path[::-1], count, execution_time

        visited.add(tuple(current_node.state.flatten()))

        current_node.expand_node()

        for child in current_node.children:
            child_state_tuple = tuple(child.state.flatten())
            if child_state_tuple not in visited:
                heapq.heappush(open_list, (child.f, child))
                visited.add(child_state_tuple)

    return None, None, None

board_numbers = np.array([[0, 8, 7], [6, 5, 4], [3, 2, 1]])
board = Node(board_numbers)

result, count, time_taken = aPathFinder(board)
if result:
    print("Tabuleiro Inicial:")
    print(board_numbers)

    print("\nTabuleiro Final:")
    print(Node.goal_state)

    print(f"\nTempo de execução: {time_taken}")
    print(f"Iterações: {count}")
else:
    print("Não foi encontrada uma solução.")


Tabuleiro Inicial:
[[0 8 7]
 [6 5 4]
 [3 2 1]]

Tabuleiro Final:
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Tempo de execução: 0.04438281059265137
Iterações: 358


## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



1. O consumo de memória depende da visitação de estados, pois ele guarda todos os estados visitados para evitar ciclos.
2. O processamento foi bom com uma quantidade elevada de iterações porém um tempo de execução bom, acredito que com uma melhora na heurística poderia melhorar a execução.